# **Check the Sentimental Analysis function**

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 235kB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 2.4MB 46.8MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
%tensorflow_version 1.x
import os
import tensorflow as tf
from konlpy.tag import Okt

import gensim
import numpy as np

TensorFlow 1.x selected.


In [0]:
class Word2Vec():
    
    def __init__(self):
        None

    def tokenize(self, doc): # 토크나이즈 하는 부분
        pos_tagger = Okt()
        return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)] # 품사 테깅하는 부분
    
    def read_data(self, filename): # 데이터 읽는 부분 
        with open(filename, 'r',encoding='utf-8') as f:
            data = [line.split('\t') for line in f.read().splitlines()]
            data = data[1:]
        return data  
    
    def Word2vec_model(self, model_name): # word2vec gensim모델을 불러들이는 부분
        model = gensim.models.word2vec.Word2Vec.load(model_name)
        return model
    
    # 한글 단어를 미리 프리트레이닝된 word2vec 모델에 lookup 테이블을 통해서 불러들이는 부분
    # 워드를 벡터로 바꾸는 부분
    def Convert2Vec(self, model_name, doc):  # Convert corpus into vectors
        word_vec = []
        model = gensim.models.word2vec.Word2Vec.load(model_name)
        for sent in doc:
            sub = []
            for word in sent: # 단어가 트레이닝된 단어장에 있으면 벡터를 불러들이고
                if(word in model.wv.vocab):
                    sub.append(model.wv[word])
                else: # 없다면 유니폼 분포를 따르는 어떤 램덤한 벡터를 생성하게 된다. 즉, OOV가 된다
                    sub.append(np.random.uniform(-0.25,0.25,300)) ## used for Out Of Vocaburaly words
            word_vec.append(sub)
        
        return np.array(word_vec)
    
    # 단어 사전에 제일 긴 단어길이에 맞춰 다른 단어들도 0으로 채워주기 위한 함수
    def Zero_padding(self, train_batch_X, Batch_size, Maxseq_length, Vector_size):
        zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
        for i in range(Batch_size):
            zero_pad[i,:np.shape(train_batch_X[i])[0],:np.shape(train_batch_X[i])[1]] = train_batch_X[i]
            
        return zero_pad
    
    # 원핫인코딩 기법
    def One_hot(self, data):
        index_dict = {value:index for index,value in enumerate(set(data))}
        result = []
        
        for value in data:
            one_hot = np.zeros(len(index_dict))
            index = index_dict[value]
            one_hot[index] = 1
            result.append(one_hot)
        
        return np.array(result)

In [0]:
class Bi_LSTM():

    def __init__(self, lstm_units, num_class, keep_prob):
        self.lstm_units = lstm_units

        with tf.variable_scope('forward', reuse=tf.AUTO_REUSE):
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob=keep_prob)

        with tf.variable_scope('backward', reuse=tf.AUTO_REUSE):
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_bw_cell, output_keep_prob=keep_prob)

        with tf.variable_scope('Weights', reuse=tf.AUTO_REUSE):
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                     dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                     initializer=tf.zeros_initializer())

    def logits(self, X, W, b, seq_len):
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,
                                                                         dtype=tf.float32,
                                                                         inputs=X, sequence_length=seq_len)
        # concat fw, bw 
        outputs = tf.concat([states[0][1], states[1][1]], axis=1) # final states
        # final state를 fully connected layer를 통해서 prediction하게 된다.
        pred = tf.matmul(outputs, W) + b # softmax
        return pred

    def model_build(self, logits, labels, learning_rate=0.001):
        with tf.variable_scope("loss"):
            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))  # Softmax loss
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)  # Adam Optimizer
        return loss, optimizer

    # Tensorboard를 통해 그래프를 만들때 쓰는 부분
    def graph_build(self):
        self.loss = tf.placeholder(tf.float32)
        self.acc = tf.placeholder(tf.float32)
        tf.summary.scalar('Loss', self.loss)
        tf.summary.scalar('Accuracy', self.acc)
        merged = tf.summary.merge_all()
        return merged

In [0]:
W2V = Word2Vec()

Batch_size = 1
Vector_size = 300
Maxseq_length = 95   ## Max length of training data
learning_rate = 0.001
lstm_units = 128
num_class = 2
keep_prob = 1.0

In [0]:
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])

In [8]:
BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
def Convert2Vec(model_name, sentence):
    
    word_vec = []
    sub = []
    model = gensim.models.word2vec.Word2Vec.load(model_name)
    for word in sentence:
        if(word in model.wv.vocab):
            sub.append(model.wv[word])
        else:
            sub.append(np.random.uniform(-0.25,0.25,300)) ## used for OOV words
    word_vec.append(sub)
    return word_vec

In [0]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
modelName = "/content/gdrive/My Drive/Colab Notebooks/GraduationProject/Bi_LSTM/BiLSTM_model_Epoch_13.ckpt"

In [11]:
sess = tf.Session()
sess.run(init)
saver.restore(sess, modelName)

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/Colab Notebooks/GraduationProject/Bi_LSTM/BiLSTM_model_Epoch_13.ckpt


In [0]:
def Grade(sentence):

    tokens = W2V.tokenize(sentence)
    
    embedding = Convert2Vec('/content/gdrive/My Drive/Colab Notebooks/GraduationProject/Word2Vec/Word2vec.model', tokens)
    zero_pad = W2V.Zero_padding(embedding, Batch_size, Maxseq_length, Vector_size)
    global sess
    result =  sess.run(tf.argmax(prediction,1), feed_dict = {X: zero_pad , seq_len: [len(tokens)] } ) 
    if(result == 1):
        print("Positive Sentence")
    else:
        print("Negative Sentence")

# **Input the Korean corpus only**

In [0]:
while(1):
    s = input("Write the Korean Sentence : ")
    if(s == str(1)):
        break
    else:
        Grade(s)

Write the Korean Sentence : 오늘 날씨가 좋아
